In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.linear_model as sk
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from random import randint
from IPython.display import display
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import preprocessing
import seaborn as sn
from sklearn.preprocessing import StandardScaler
from sklearn.semi_supervised import LabelPropagation
import warnings
warnings.filterwarnings('ignore')

In [2]:
def semi_supervised(data_df,noise_frac,K,label_column):
    def detected_indices_temp(data,p,col,model):
        data['original'] = data[col]
        class0 = data.loc[data[col]!=1,:]
        class1 = data.loc[data[col]!=0,:]
        def noise(data):
            sample = data.sample(frac = noise_frac)
            for i in list(sample.index):
                data[col][i] = int(not data[col][i])
            return list(sample.index)    
        noise0 = noise(class0)
        noise1 = noise(class1)
        noisy_indices = noise0+noise1
        mislabeled = pd.concat([class0,class1])
        indices = list(mislabeled.index)

        # User Optimal Number of Cluster
        km = KMeans(n_clusters = p)
        km.fit(mislabeled.loc[:,mislabeled.columns!=col])
        label = km.labels_
        mislabeled['cluster'] = label
        mislabeled['original'] = data['original']
        #OneHotEncoding
        mislabeled = pd.get_dummies(mislabeled,columns=['cluster'])


        #RandomForestClassifier
        x_train,x_test,y_train,y_test = train_test_split(mislabeled.loc[:,mislabeled.columns!=col],mislabeled[col],test_size = 0.38)
        rf = model()
        rf.fit(x_train,y_train)
        predict = rf.predict(mislabeled.loc[:,mislabeled.columns!=col])
        mislabeled['predicted'] = predict
        detected = list()
        cnt = 0

        ###########################################################################
    #     print((mislabeled.loc[:,df.columns != [[col,'predicted']]],mislabeled[col]))

        for i in indices:
            if mislabeled[col][i] != mislabeled['predicted'][i]:
                mislabeled[col][i] = -1

        #Semisupervised
    #     print(mislabeled[mislabeled.columns.difference([col,'predicted'])],mislabeled[col])
        lp = LabelPropagation()
        lp.fit(mislabeled[mislabeled.columns.difference([col,'predicted','original'])],mislabeled[col])
        arr = lp.predict(mislabeled[mislabeled.columns.difference([col,'predicted','original'])])

        mislabeled['semi'] = arr

        ############################################################
        for i in indices:
            if mislabeled[col][i] != mislabeled['semi'][i]:
                detected.append(i)
        bad_change =  0
        #detected Noise (Comparing with mislabled Data label)
        for i in detected:
            if i in noisy_indices:
                cnt+=1
            else:
                bad_change+=1

#         return pd.DataFrame(data={'Total Mislabeled':[len(noisy_indices)], 
#                                   'Total Mismatch':[len(detected)], 
#                                   'True Identified labels(Actual Noise)':[cnt], 
#                                   'False identified labels(Not Noise fliped)':[bad_change], 
#                                   'Total Noise Ratio':len(noisy_indices)/data.shape[0], 
#                                   'True Identified Ratio(Actual Noise detected ratio)':cnt/len(noisy_indices)}
#                            )
        
        #
        #  We Are FLIPING the data
        #
        #
        
        n_n = cnt
        n_p = len(noisy_indices) - cnt
        p_n = bad_change 
        p_p = data_df.shape[0] - len(noisy_indices) - bad_change
        pure_per = 100 * (p_p+n_n)/(data_df.shape[0])
        noise_per = 100 * (n_p + p_n)/(data_df.shape[0])
        cleaned_per = 100 *(n_n - p_n)/data_df.shape[0]

        #
        #  We Are deleting the data
        #
        #
        
#         n_n = cnt
#         n_p = len(noisy_indices) - cnt
#         p_n = bad_change 
#         p_p = data_df.shape[0] - len(noisy_indices) - bad_change
#         pure_per = 100 * (p_p)/(data_df.shape[0] - len(detected))
#         noise_per = 100 * (n_p)/(data_df.shape[0] - len(detected))
#         cleaned_per = 100 * (n_n)/data_df.shape[0]
        
        
        return pd.DataFrame(data={'# Obeservation in Culprit':[len(detected)], 
                                  'Noise to Noise':n_n, 
                                  'Pure to Noise': p_n, 
                                  'Noise to Pure': n_p, 
                                  'Pure to Pure': p_p, 
                                  'Pure %': pure_per,
                                  'Noise %': noise_per,
                                  'Cleaned %': cleaned_per
                                 }
                           )
    
    
        
    
    


    models = [SVC,RandomForestClassifier,GaussianNB,LogisticRegression,DecisionTreeClassifier]
    #result = pd.DataFrame(data={'Total Noise':[], 'Total Detected labels':[], 'True Identified labels':[], 'False identified labels':[], 'Total Noise Ratio':[], 'True Identified Ratio':[]})
    result = pd.DataFrame()
    for model in models:
        temp = detected_indices_temp(data_df,K,label_column,model)
        result = result.append(temp, ignore_index=True)
    result['Model']=pd.Series(["SVC","Random Forest","Naive Bayes","Logistic Regression","Decision Tree"])
    return result





In [3]:
# K = 14
# data_df = pd.read_csv('../SPECT.train',header=None)
# label_column = 0


In [4]:
df = pd.read_csv('../heart-disease.data',header=None, names=['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','label'])
label = df['label']
df.replace(r'?',np.nan,inplace=True)
df.fillna(df.median(),inplace=True)
df_std = StandardScaler().fit_transform(df)
df = pd.DataFrame(df_std)
df.head()
df.columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','label']
df['label'] = label
df['label'] = df['label'].map({1:1,2:1,3:1,4:1,0:0})
df.head()
data_df = df
label_column = 'label'
K = 2

In [ ]:
# data_df = pd.read_csv('../Binary Classified Datasets/ionosphere.csv')
# label_column = 'g'
# K = 2

In [ ]:
l = []
for i in range(1,6):
    l.append(semi_supervised(data_df,i/10,K,label_column))

In [ ]:
final=pd.concat(l)

In [ ]:
final.to_csv('~/Heart')

In [ ]:
data_df.shape[0]